In [16]:
import intersect
import pandas as pd
import geopandas as gpd
import fiona.crs

In [4]:
raw_people = pd.read_csv("/Users/asiega/Desktop/CVHpeople.csv")

In [5]:
raw_people.head()

,Internal Contact ID,Latitude,Longitude
0,5,40.687482,-73.963384
1,226,40.769909,-73.992111
2,228,40.746794,-73.982509
3,230,40.823687,-73.868344
4,232,40.760444,-73.973270


In [8]:
people = intersect.people_df("/Users/asiega/Desktop/CVHpeople.csv")

In [9]:
people.head()

,Internal Contact ID,Latitude,Longitude,geometry
0,5,40.687482,-73.963384,POINT (-73.96338399999999 40.687482)
1,226,40.769909,-73.992111,POINT (-73.99211099999999 40.76990900000001)
2,228,40.746794,-73.982509,POINT (-73.9825089 40.7467938)
3,230,40.823687,-73.868344,POINT (-73.86834350000001 40.8236871)
4,232,40.760444,-73.973270,POINT (-73.97327 40.760444)


In [27]:
raw_places = gpd.read_file("/Users/asiega/Desktop/NYCHA.geojson")

In [30]:
raw_places.head()

,BOROUGH,CUR_UNIT11,DEVELOPMEN,NONRES_BLD,RES_BLDG,TDS_NUM,TOT_POP11,geometry
0,VARIOUS,16,FHA REPOSSESSED HOUSES (GROUP III),0,18,213,40,"(POLYGON ((1002120 179553, 1002021 179544, 100..."
1,VARIOUS,19,FHA REPOSSESSED HOUSES (GROUP IV),0,22,226,60,"(POLYGON ((1003910 180501, 1003835 180437, 100..."
2,VARIOUS,40,FHA REPOSSESSED HOUSES (GROUP V),0,50,260,103,"(POLYGON ((1005655 178293, 1005640 178281, 100..."
3,QUEENS,9,FHA REPOSSESSED HOUSES (GROUP VI),0,14,273,22,"(POLYGON ((1046847 181178, 1046756 181125, 104..."
4,QUEENS,11,FHA REPOSSESSED HOUSES (GROUP VII),0,17,274,22,"(POLYGON ((1048186 180368, 1048087 180357, 104..."


In [36]:
places = raw_places.to_crs(fiona.crs.from_epsg(2263))

In [41]:
places["geometry"].head()
# if you explore here, you notice that with to_crs, every polygon value becomes "inf"

0    (POLYGON ((inf inf, inf inf, inf inf, inf inf,...
1    (POLYGON ((inf inf, inf inf, inf inf, inf inf,...
2    (POLYGON ((inf inf, inf inf, inf inf, inf inf,...
3    (POLYGON ((inf inf, inf inf, inf inf, inf inf,...
4    (POLYGON ((inf inf, inf inf, inf inf, inf inf,...
Name: geometry, dtype: object